# Tutorial: Spatiotemporal prediction/analysis in Python using GPyTorch (Gaussian Processes)

This notebook covers a simple example of using GPyTorch to perform spatiotemporal prediction. We will use two example datasets: One is a list of Airbnb listings in Austin, TX

We will use the Airbnb dataset to predict the price of a listing given a set of predictors. 

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import osmnx as ox
import matplotlib.pyplot as plt
from scipy import stats
import torch
import gpytorch
from pysal.model import spreg
from pysal.lib import weights
from utils import GP

C:\Users\ekino\AppData\Local\Temp\ipykernel_26676\357393232.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
torch.cuda.is_available()

False

Let's read the Airbnb data and OSM data for Austin, TX.

In [3]:
# Read listings
fp = "data/listings.csv"
df = pd.read_csv(fp)
df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [4]:
df.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,72635,https://www.airbnb.com/rooms/72635,20151107173015,2015-11-08,"3 Private Bedrooms, SW Austin",Conveniently located 10-15 from downtown in SW...,"We have three spare bedrooms, each with a quee...",Conveniently located 10-15 from downtown in SW...,none,Location and convenience are key. Easy access...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,0.02
1,5386323,https://www.airbnb.com/rooms/5386323,20151107173015,2015-11-07,Cricket Trailer,Rent this cool concept trailer that has everyt...,Rental arrangements for this trailer allows yo...,Rent this cool concept trailer that has everyt...,none,We're talking about wherever you'd like in the...,...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN


In [5]:
# Define the place query
place = {'city': 'Austin'}

# Get the place polygon
boundaries = ox.geocode_to_gdf(place, buffer_dist=5000)

# Can see this on the map using the following command
#boundaries.explore()

Let’s convert the Airbnb data into GeoDataFrame based on the longitude and latitude columns and filter the data
geographically based on Austin boundaries:

In [6]:
# Create a GeoDataFrame
df["geometry"] = gpd.points_from_xy(df["longitude"], df["latitude"])
df = gpd.GeoDataFrame(df, crs="epsg:4326")

# Filter data geographically
df = gpd.sjoin(df, boundaries[["geometry"]])
df = df.reset_index(drop=True)

# Check the first rows
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,geometry,index_right
0,72635,https://www.airbnb.com/rooms/72635,20151107173015,2015-11-08,"3 Private Bedrooms, SW Austin",Conveniently located 10-15 from downtown in SW...,"We have three spare bedrooms, each with a quee...",Conveniently located 10-15 from downtown in SW...,none,Location and convenience are key. Easy access...,...,NaN,NaN,f,moderate,f,f,1,0.02,POINT (-97.88431 30.20282),0
1,5386323,https://www.airbnb.com/rooms/5386323,20151107173015,2015-11-07,Cricket Trailer,Rent this cool concept trailer that has everyt...,Rental arrangements for this trailer allows yo...,Rent this cool concept trailer that has everyt...,none,We're talking about wherever you'd like in the...,...,NaN,NaN,f,moderate,f,f,1,NaN,POINT (-97.90068 30.19941),0
2,8826517,https://www.airbnb.com/rooms/8826517,20151107173015,2015-11-07,Private room 1 in South Austin,"Upstairs, private, 12ft x 13 1/2ft room. Priv...",NaN,"Upstairs, private, 12ft x 13 1/2ft room. Priv...",none,NaN,...,NaN,NaN,f,flexible,f,f,2,NaN,POINT (-97.86448 30.16850),0
3,8828616,https://www.airbnb.com/rooms/8828616,20151107173015,2015-11-08,Private room 2 in South Austin,"Upstairs, private, 11ft x 13 1/2ft room. Priv...",NaN,"Upstairs, private, 11ft x 13 1/2ft room. Priv...",none,NaN,...,NaN,NaN,f,flexible,f,f,2,NaN,POINT (-97.86487 30.16862),0
4,8536913,https://www.airbnb.com/rooms/8536913,20151107173015,2015-11-08,Brand-New 3BR Austin Home,Brand-new 3BR/2BA Austin home with landscaped ...,Feel instantly at home at our brand new 3BR/2B...,Brand-new 3BR/2BA Austin home with landscaped ...,none,Entertainment and activities are plentiful her...,...,NaN,NaN,f,strict,f,f,2,NaN,POINT (-97.88832 30.16943),0


Let's take a look at the price column:

In [7]:
df['price'].head()

0    $300.00
1     $99.00
2    $100.00
3    $100.00
4    $599.00
Name: price, dtype: object

In [8]:
# Remove the dollar sign, convert values to floats
df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)

C:\Users\ekino\AppData\Local\Temp\ipykernel_26676\486199180.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)


In [9]:
# Here the tooltip parameter specifies which attributes are shown when hovering on top of the points
# The vmax parameter specifies the maximum value for the colormap (here, all 1000 dollars and above are combined)

#df.explore(column="price", cmap="Reds", scheme="quantiles", k=4, tooltip=["name", "price"], vmax=1000,
#tiles="CartoDB Positron")

## Baseline Method: Linear Regression (nonspatial)

Before introducing explicitly spatial methods, we will run a simple linear regression model. This will allow us, on the one hand,
set the main principles of hedonic modeling and how to interpret the coefficients, which is good because the spatial models
will build on this; and, on the other hand, it will provide a baseline model that we can use to evaluate how meaningful the
spatial extensions are.

The core of a linear regression is to explain a given variable--the price $P_i$ of a listing $i$ on AirBnb--as a linear function of a set of other characteristics $X_i$.

$$ \ln(P_i) = \alpha + \beta X_i + \epsilon_i $$

For several reasons, it is common to represent the price logarithmically. Furthermore, since this is a probabilistic model, we add an error term $\epsilon_i$ that is assumed to be normally distributed (i.i.d.). 

Let's consider the following set of explanatory variables for each listing:

In [10]:
explanatory_vars = ['host_listings_count', 'bathrooms', 'bedrooms', 'beds', 'guests_included', 'pool']

Additionally, we are going to derive a new feature of a listing from the amenities variable. Let us construct a variable that takes
1 if the listed property has a pool and 0 otherwise:

In [11]:
def has_pool(a):
    if 'Pool' in a:
        return 1
    else:
        return 0
 
df['pool'] = df['amenities'].apply(has_pool)

Let’s then calculate the logarithmic value of the price:

In [12]:
df["log_price"] = np.log(df["price"] + 0.000001)

Let's check for missing values in either our dependent or predictor variables:

In [13]:
all_model_attributes = ["price"] + explanatory_vars
has_nans = False
for attr in all_model_attributes:
    if df[attr].hasnans:
        has_nans = True
print("Has missing values:", has_nans)

Has missing values: True


Let's remove missing values:

In [14]:
df = df.dropna(subset=all_model_attributes).copy()

Let's fit the regression

In [15]:
m1 = spreg.OLS(df[['log_price']].values, df[explanatory_vars].values, name_y = 'log_price', name_x = explanatory_vars)
print(m1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :   log_price                Number of Observations:        5760
Mean dependent var  :      5.1955                Number of Variables   :           7
S.D. dependent var  :      0.9457                Degrees of Freedom    :        5753
R-squared           :      0.4038
Adjusted R-squared  :      0.4032
Sum squared residual:    3070.660                F-statistic           :    649.4612
Sigma-square        :       0.534                Prob(F-statistic)     :           0
S.E. of regression  :       0.731                Log likelihood        :   -6361.437
Sigma-square ML     :       0.533                Akaike info criterion :   12736.874
S.E of regression ML:      0.7301                Schwarz criterion     :   12783.484

-----------------------------------------------------------------------------

Results are largely unsurprising, but nonetheless reassuring. Both an extra bedroom and an extra bathroom increase the final
price around 30%. Accounting for those, an extra bed pushes the price about 2%, while the availability of a pool increases the price by 9%.

However, there might be non-linear relationships between the price and the number of bedrooms, bathrooms, beds, etc. We will now use Gaussian processes to model these relationships. 

### GPyTorch: Gaussian Processes in PyTorch

We will first separate our data into a training and testing set. Later, we will see how to do this more efficiently using cross-validation, and we will employ batched GP models to test each split in parallel. A couple of other things to note:
* If your device has a GPU, you can use it to speed up the training process. To do so, you need a CUDA-enabled PyTorch installation. You can check if your device has a CUDA-enabled GPU by running `torch.cuda.is_available()`. More info here: https://pytorch.org/get-started/locally/#windows-anaconda

In [16]:
# Separate data into training and test sets
train = df.sample(frac=0.8, random_state=0)
test = df.drop(train.index)

In [21]:
# Convert the data to torch tensors
X = torch.tensor(train[explanatory_vars].values, dtype=torch.float)
y = torch.tensor(train["log_price"].values, dtype=torch.float)

# Enable CUDA (to run on GPU)
#X = X.cuda()
#y = y.cuda()

The most important aspect of any GP model is the kernel, a function that defines the covariance between two points in the input space. The covariance is a measure of how similar two points are. The kernel is a function that takes two points as input and returns the covariance between them. 

Here, we will use the rational quadratic kernel, which is a generalization of the squared exponential kernel. The rational quadratic kernel is defined as:

$$ \textbf{K}_{RQ}(x, x') = \sigma^2 \left(1 + \frac{(x - x')^2}{2 \alpha l^2}\right)^{-\alpha} $$

where $\sigma^2$ is the variance, $\alpha$ is the shape parameter, and $l$ is the lengthscale. This kernel is equivalent to adding together many squared exponential (SE) kernels with different lengthscales. So, GP priors with this kernel expect to see functions which vary smoothly across many lengthscales. The parameter $\alpha$ determines the relative weighting of large-scale and small-scale variations. When $\alpha \rightarrow \infty$, the RQ is identical to the SE.

Notice however that the above kernel is only defined for a one-dimensional vector (with $x$ and $x'$ being two separate inputs). In our case, we have a multi-dimensional input space, so we need to define a kernel that can handle this. Multiplying kernels is a standard way to fit higher-dimensional data. In our case, we will use the product of many RQ kernels, one for each dimension of the input space.

$$ \textbf{K}_{RQ-ARD}(x, x') = \prod_{i=1}^D \textbf{K}_{RQ}(x_i, x_i') $$

where $D$ is the number of dimensions of the input space. In the literature, this is referred to as an "automatic relevance determination" (ARD) kernel, so named because the lengthscale of each dimension determines the relative importance of that dimension in the model (i.e., the higher the lengthscale, the more important that dimension is). You can think of it as the p-value of a predictor in a linear regression model.

In [18]:
# Define the kernel
kernel = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RQKernel(ard_num_dims=X.shape[1]))

# Define the model class
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, X, y, likelihood):
        super(ExactGPModel, self).__init__(X, y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = kernel

    def forward(self, X):
        mean_x = self.mean_module(X)
        covar_x = self.covar_module(X)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(X, y, likelihood)

In [20]:
ls, mll = GP.training(model, X, y)

 40%|████      | 81/200 [03:26<05:03,  2.55s/it, Loss=1.027, noise=0.427, lengthscale=['6.077', '5.239', '2.628', '6.469', '3.609', '6.927']]
